# In this project, we solve the problem of picking up shipments from locations. The data is real-life, and obtained from the following link:
https://www.kaggle.com/datasets/hemanthboddapu/sample-data-shipments-vehicle-routing-simulation

# 15 vehicles, 3 each from 5 depots, start at their designated depot and pick up shipments from the give locations. 

## 1. Import

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd gdrive/MyDrive/232E-SocialNetworks/Project4

In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.5 MB 464 kB/s 
     |████████████████████████████████| 407 kB 40.1 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220527125636 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.1 which is incompatible.
tensorflow-metadata 1.8.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.1 which is incompatible.
googleapis-common-protos 1.56.2 requires protobuf<4.0.0dev,>=3.15.0, but you have protobuf 4.21.1 which is incompatible.
google-api-core 1.31.6 requires protobuf<4.0.0dev,>=3.12.0; python_version > "3", but you have p

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import numpy as np
import random
import os
import sys
import itertools
from geopy import distance

## 2. EDA

In [ ]:
df = pd.read_csv("Dataset.csv")

In [ ]:
df

,Date,City,Latitude,Longitude,Weight
0,04-01-2020,Delhi,28.6600,77.2300,6.025515
1,25-04-2020,Mumbai,18.9667,72.8333,14.766201
2,10-03-2020,Delhi,28.6600,77.2300,14.624218
3,15-05-2020,Delhi,28.6600,77.2300,12.139311
4,15-08-2020,Indore,22.7206,75.8472,6.468847
...,...,...,...,...,...
920,23-06-2020,Ulhasnagar,19.2167,73.1500,6.918289
921,10-12-2019,Hyderabad,17.3667,78.4667,5.793844
922,31-01-2020,Cawnpore,26.4725,80.3311,5.701604
923,20-10-2020,Pune,18.5196,73.8553,12.544136


In [ ]:
cities = df['City'].unique()
cities.sort()

In [ ]:
print(cities)
print(len(cities))

['Agra' 'Ahmadabad' 'Ajmer' 'Aligarh' 'Allahabad' 'Amravati' 'Amritsar'
 'Asansol' 'Aurangabad' 'Bangalore' 'Bareilly' 'Bezwada' 'Bhavnagar'
 'Bhayandar' 'Bhilai' 'Bhiwandi' 'Bhopal' 'Bhubaneshwar' 'Bikaner'
 'Cawnpore' 'Chanda' 'Chandigarh' 'Chennai' 'Chinchvad' 'Coimbatore'
 'Cuttack' 'Dehra Dun' 'Delhi' 'Dhanbad' 'Dhulia' 'Dispur' 'Durgapur'
 'Faridabad' 'Gaya' 'Ghaziabad' 'Gorakhpur' 'Gulbarga' 'Guntur' 'Guwahati'
 'Gwalior' 'Haora' 'Hubli' 'Hyderabad' 'Indore' 'Jabalpur' 'Jaipur'
 'Jalandhar' 'Jalgaon' 'Jammu' 'Jamnagar' 'Jamshedpur' 'Jodhpur' 'Kalyan'
 'Kochi' 'Kolhapur' 'Kolkata' 'Kota' 'Kurnool' 'Lucknow' 'Ludhiana'
 'Madurai' 'Malegaon' 'Mirzapur' 'Moradabad' 'Mumbai' 'Mysore' 'Nagpur'
 'Nanded' 'Nasik' 'Nellore' 'Patna' 'Pune' 'Raipur' 'Rajkot' 'Ranchi'
 'Raurkela' 'Saharanpur' 'Salem' 'Shimoga' 'Srinagar' 'Surat' 'Thane'
 'Thiruvananthapuram' 'Tinnevelly' 'Tiruppur' 'Trichinopoly' 'Ujjain'
 'Ulhasnagar' 'Vadodara' 'Varanasi' 'Vishakhapatnam' 'Warangal']
92


## 3. Set up Problem

In [ ]:
# Get demands by averaging the weights of shipments bound for each city over all time in the original dataset

dataset = df.groupby("City")["Weight"].mean().reset_index()

In [ ]:
dataset

,City,Weight
0,Agra,6.347402
1,Ahmadabad,7.935774
2,Ajmer,3.955014
3,Aligarh,7.409956
4,Allahabad,7.944810
...,...,...
87,Ulhasnagar,8.479798
88,Vadodara,6.090675
89,Varanasi,10.642147
90,Vishakhapatnam,10.355121


In [ ]:
# Import latitudes and longitudes from original dataset to our dataset

latitude = {}
longitude = {}
for city in cities:
    latitude[city] = df.loc[df.City == city]["Latitude"].values[0]
    longitude[city] = df.loc[df.City == city]["Longitude"].values[0]

dataset["Latitude"] = dataset["City"].map(latitude)
dataset["Longitude"] = dataset["City"].map(longitude)

In [ ]:
# Contains City, Demand, Latitude, Longitude

dataset

,City,Weight,Latitude,Longitude
0,Agra,6.347402,27.1800,78.0200
1,Ahmadabad,7.935774,23.0300,72.5800
2,Ajmer,3.955014,26.4680,74.6390
3,Aligarh,7.409956,27.8800,78.0800
4,Allahabad,7.944810,25.4550,81.8400
...,...,...,...,...
87,Ulhasnagar,8.479798,19.2167,73.1500
88,Vadodara,6.090675,22.3000,73.2000
89,Varanasi,10.642147,25.3189,83.0128
90,Vishakhapatnam,10.355121,17.7333,83.3167


In [ ]:
# Find distances between each pair of cities and store it in data["distance_matrix"]

data = {}
data["distance_matrix"] = [[0 for _ in range(len(dataset))] for _ in range(len(dataset))]
for i in range(len(dataset)):
    for j in range(len(dataset)):
        if i != j:
            dist = distance.distance((dataset.iloc[i, 2], dataset.iloc[i, 3]), (dataset.iloc[j, 2], dataset.iloc[j, 3])).km
            data["distance_matrix"][i][j] = dist
            data["distance_matrix"][j][i] = dist

In [ ]:
# Set the 5 major cities as depots, so get their indices

print(np.where(dataset["City"] == "Delhi"))
print(np.where(dataset["City"] == "Mumbai"))
print(np.where(dataset["City"] == "Bangalore"))
print(np.where(dataset["City"] == "Hyderabad"))
print(np.where(dataset["City"] == "Kolkata"))

(array([27]),)
(array([64]),)
(array([9]),)
(array([42]),)
(array([55]),)


In [ ]:
capacity = [50]
data['num_vehicles'] = 15                                       # A total of 15 vehicles, 3 for each depot
vehicle_plan = sorted([27, 64, 9, 42, 55] * 3)
data["starts"] = vehicle_plan                   # Set start and end for vehicle routes(depots)
data["ends"] = vehicle_plan
data["demands"] = dataset["Weight"].to_list()
data['vehicle_capacities'] = capacity * data['num_vehicles']        # set capacities of each vehicle
data['city'] = dataset['City']
print(data.keys())

dict_keys(['distance_matrix', 'num_vehicles', 'starts', 'ends', 'demands', 'vehicle_capacities', 'city'])


In [ ]:
len(data['demands'])

92

In [ ]:
def print_solution(data, manager, routing, solution, isprint=True):
    #print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id + 1}:\n'
        route_distance = 0
        route_load = 0
        count = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            # skip accounting for route_load at the depot
            if count != 0:
                route_load += data['demands'][node_index]
            city_visit=data['city'][node_index]
            plan_output += ' {}, Load({:.2f}) -> '.format(city_visit, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            count += 1
        end_city = data['city'][int(manager.IndexToNode(index))]
        plan_output += ' {}, Load({:.2f})\n'.format(end_city,
                                                 route_load)
        plan_output += 'Distance of the route: {}km\n'.format(route_distance)
        plan_output += 'Load of the route: {:.2f}\n'.format(route_load)
        if isprint: print(plan_output)
        total_distance += route_distance
        total_load += route_load

    if isprint:
        print('Total distance of all routes: {}km'.format(total_distance))
        print('Total load of all routes: {:.2f}'.format(total_load))

    return total_distance

## 4. Find Routes

In [ ]:
def find_route(isprint):

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['starts'], data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        total_distance = print_solution(data, manager, routing, solution,isprint)

    return total_distance 
    

baseline_distance = find_route(isprint=True)

Route for vehicle 1:
 Bangalore, Load(0.00) ->  Salem, Load(5.72) ->  Chennai, Load(13.14) ->  Nellore, Load(22.41) ->  Guntur, Load(31.58) ->  Kurnool, Load(39.76) ->  Bangalore, Load(39.76)
Distance of the route: 1383km
Load of the route: 39.76

Route for vehicle 2:
 Bangalore, Load(0.00) ->  Tiruppur, Load(7.77) ->  Coimbatore, Load(12.26) ->  Kochi, Load(18.45) ->  Thiruvananthapuram, Load(27.13) ->  Tinnevelly, Load(32.89) ->  Madurai, Load(41.05) ->  Trichinopoly, Load(46.41) ->  Bangalore, Load(46.41)
Distance of the route: 1174km
Load of the route: 46.41

Route for vehicle 3:
 Bangalore, Load(0.00) ->  Mysore, Load(7.03) ->  Shimoga, Load(13.67) ->  Malegaon, Load(18.94) ->  Dhulia, Load(24.72) ->  Indore, Load(31.30) ->  Jalgaon, Load(38.48) ->  Aurangabad, Load(46.32) ->  Bangalore, Load(46.32)
Distance of the route: 2475km
Load of the route: 46.32

Route for vehicle 4:
 Delhi, Load(0.00) ->  Moradabad, Load(5.35) ->  Bareilly, Load(11.43) ->  Lucknow, Load(20.24) ->  Cawnpor

## 5. Find Optimal Vehicle Plan

In [ ]:
minimum_vehicle = [27, 64, 9, 42, 55]
vehicle_plans = list(itertools.combinations_with_replacement([27, 64, 9, 42, 55], 10))
vehicle_plans = [list(vehicle_plan) for vehicle_plan in vehicle_plans]

for vehicle_combination in vehicle_plans:
    vehicle_combination.extend(minimum_vehicle)

print(len(vehicle_plans))

1001


In [ ]:
# Baseline distance = distance for 3 vehicles per depot:
print(f"Baseline distance: {baseline_distance:.2f}")

Baseline distance: 21233.00


In [ ]:
def find_opt_path(vehicle_plans):
    costs = []
    for vehicle_plan in vehicle_plans:
        data['starts'] = vehicle_plan
        data['ends'] = vehicle_plan
        distance = find_route(isprint=False)
        distance_diff = distance - baseline_distance
        costs.append(distance_diff)

    arr_costs = np.array(costs)
    optimal_plan = vehicle_plans[np.argmin(arr_costs)]
    print(f"The optimal vehicle plan is: {optimal_plan}")
    return optimal_plan

optimal_plan = sorted(find_opt_path(vehicle_plans))

The optimal vehicle plan is: [27, 27, 27, 27, 64, 64, 42, 42, 55, 55, 27, 64, 9, 42, 55]


In [ ]:
# Print the optimal vehicle plan's route
data['starts'] = optimal_plan
data['ends'] = optimal_plan
_ = find_route(isprint=True)

Route for vehicle 1:
 Bangalore, Load(0.00) ->  Salem, Load(5.72) ->  Trichinopoly, Load(11.07) ->  Madurai, Load(19.23) ->  Tinnevelly, Load(24.99) ->  Thiruvananthapuram, Load(33.67) ->  Kochi, Load(39.86) ->  Coimbatore, Load(44.35) ->  Bangalore, Load(44.35)
Distance of the route: 1153km
Load of the route: 44.35

Route for vehicle 2:
 Delhi, Load(0.00) ->  Bikaner, Load(11.62) ->  Jodhpur, Load(19.91) ->  Ajmer, Load(23.87) ->  Jaipur, Load(31.08) ->  Agra, Load(37.43) ->  Aligarh, Load(44.84) ->  Delhi, Load(44.84)
Distance of the route: 1289km
Load of the route: 44.84

Route for vehicle 3:
 Delhi, Load(0.00) ->  Gwalior, Load(8.69) ->  Jabalpur, Load(19.65) ->  Bhilai, Load(28.89) ->  Raipur, Load(37.86) ->  Cawnpore, Load(45.43) ->  Delhi, Load(45.43)
Distance of the route: 1936km
Load of the route: 45.43

Route for vehicle 4:
 Delhi, Load(0.00) ->  Moradabad, Load(5.35) ->  Bareilly, Load(11.43) ->  Bhopal, Load(18.58) ->  Jalgaon, Load(25.76) ->  Dhulia, Load(31.53) ->  Indore